<a href="https://colab.research.google.com/github/Bertrand-gbt/Projet/blob/master/AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Commençons par importer nos images.

Ici nous testons notre modèle avec des données qui sont des vraies photos ou alors des photos retoucher, le but étant de déterminer si elles ont été retouchées.

In [0]:
from skimage import io
import glob

In [0]:
import numpy as np

In [0]:
X_true = []
X_false = []

In [0]:
for filename in glob.glob('/content/drive/My Drive/Projet/True_trainset/*.png'):
    im = io.imread(filename)
    X_true.append(im)

In [0]:
X_true = np.array(X_true)

In [55]:
Y_true = np.array([[1,0] for k in range(1081)])
Y_true.shape

(1081, 2)

In [0]:
for filename in glob.glob('/content/drive/My Drive/Projet/False_trainset/*.png'):
    im = io.imread(filename)
    X_false.append(im)

In [43]:
X_false = np.array(X_false)
X_false.shape

(960, 227, 227, 3)

In [56]:
Y_false = np.array([[0,1] for k in range(960)])
Y_false.shape

(960, 2)

In [57]:
X = np.concatenate((X_true,X_false),axis=0)
X.shape

(2041, 227, 227, 3)

In [58]:
Y = np.concatenate((Y_true,Y_false),axis=0)
Y.shape

(2041, 2)

Maintenant nous allons créer le modèle et par la suite l'entraîner sur nos images.

In [0]:
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical

In [0]:
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(227,227,3), kernel_size=(11,11), strides=(4,4), padding="valid", activation = "relu"))

# Max Pooling
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2), padding="valid"))

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding="same", activation = "relu"))

# Max Pooling
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2), padding="valid"))

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="same", activation = "relu"))

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="same", activation = "relu"))

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding="same", activation = "relu"))

# Max Pooling
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2), padding="valid"))

# Passing it to a Fully Connected layer
model.add(Flatten())
# 1st Fully Connected Layer
model.add(Dense(units = 9216, activation = "relu"))

# 2nd Fully Connected Layer
model.add(Dense(units = 4096, activation = "relu"))

# 3rd Fully Connected Layer
model.add(Dense(4096, activation = "relu"))

# Output Layer
model.add(Dense(2, activation = "softmax")) #As we have two classes

In [0]:
from keras.optimizers import Adam
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [74]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 55, 55, 96)        34944     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 27, 27, 256)       614656    
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 13, 13, 384)       885120    
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 13, 13, 384)       1327488   
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 13, 13, 256)      

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [75]:
model.fit(x=X_train,     
            y=y_train, 
            batch_size=50, 
            epochs=20, 
            verbose=1, 
            validation_data=(X_test,y_test),
            shuffle=True)

Train on 1632 samples, validate on 409 samples
Epoch 1/20
1632/1632 [==============================] - 6s 3ms/step - loss: 7.4983 - acc: 0.5214 - val_loss: 7.2906 - val_acc: 0.5477
Epoch 2/20
1632/1632 [==============================] - 3s 2ms/step - loss: 7.6541 - acc: 0.5251 - val_loss: 7.2906 - val_acc: 0.5477
Epoch 3/20
1632/1632 [==============================] - 3s 2ms/step - loss: 7.6541 - acc: 0.5251 - val_loss: 7.2906 - val_acc: 0.5477
Epoch 4/20
1632/1632 [==============================] - 3s 2ms/step - loss: 7.6541 - acc: 0.5251 - val_loss: 7.2906 - val_acc: 0.5477
Epoch 5/20
1632/1632 [==============================] - 3s 2ms/step - loss: 7.6541 - acc: 0.5251 - val_loss: 7.2906 - val_acc: 0.5477
Epoch 6/20
1632/1632 [==============================] - 3s 2ms/step - loss: 7.6541 - acc: 0.5251 - val_loss: 7.2906 - val_acc: 0.5477
Epoch 7/20
1632/1632 [==============================] - 3s 2ms/step - loss: 7.6541 - acc: 0.5251 - val_loss: 7.2906 - val_acc: 0.5477
Epoch 8/20
1632

In [76]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

y_pred = model.predict(X_test)

print('Score R²: %.4f' % r2_score(y_test,y_pred))
print('Erreur quadratique moyenne : %.4f' % mean_squared_error(y_test,y_pred))
print('Erreur absolue moyenne : %.4f' % mean_absolute_error(y_test,y_pred))

Score R²: -0.8259
Erreur quadratique moyenne : 0.4523
Erreur absolue moyenne : 0.4523
